# アノテーションテキストの作り方
https://reveltb.com/posts/ikath/coco-api/

In [ ]:
!pip install cython
%cd /content
!rm -rf cocoapi
!git clone https://github.com/cocodataset/cocoapi.git
%cd /content/cocoapi/PythonAPI

/content
Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 3.44 MiB/s, done.
Resolving deltas: 100% (576/576), done.
/content/cocoapi/PythonAPI


In [ ]:
!make
!make install

python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/pycocotools
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -I../common -I/usr/include/python3.6m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.6/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
../common/maskApi.c: In function ‘rleDecode’:
../common/maskApi.c:

# install pycocotools to the Python site-packages
python setup.py build_ext install
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/pycocotools
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -I../common -I/usr/include/python3.6m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.6/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
../common/maskApi.c: In function ‘rleDecode’:
../common/maskApi.c:46:7: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
       ^~~
../common/maskApi.c:46:49: note: ...this statement, but the latter is misleadingly indented 

In [ ]:
#class.txtを作る
!touch class.txt 


IndentationError: ignored

In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import numpy as np
import skimage.io as io
import pylab
import os
import shutil

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

ModuleNotFoundError: ignored

In [ ]:
dataDir='..'
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)

# メモリ使用量、
https://qiita.com/fockl/items/ada469c138900caaf0a8

データ
https://www.pexels.com/ja-jp/search/videos/%E4%BA%BA/

In [ ]:
!nvidia-smi

Tue Dec  8 01:10:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    39W / 300W |   9053MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-cp36-none-any.whl size=30181 sha256=99b331084fa57ac913229028e91ec52535d9de2aa35f4d6f61a2d353c3b4a474
  Stored in directory: /root/.cache/pip/wheels/02/e4/0b/aaab481fc5dd2a4ea59e78bc7231bb6aae7635ca7ee79f8ae5
Successfully built memory-profiler


In [ ]:
import subprocess
import shlex

def gpuinfo():
    """
    Returns size of total GPU RAM and used GPU RAM.

    Parameters
    ----------
    None

    Returns
    -------
    info : dict
        Total GPU RAM in integer for key 'total_MiB'.
        Used GPU RAM in integer for key 'used_MiB'.
    """

    command = 'nvidia-smi -q -d MEMORY | sed -n "/FB Memory Usage/,/Free/p" | sed -e "1d" -e "4d" -e "s/ MiB//g" | cut -d ":" -f 2 | cut -c2-'
    commands = [shlex.split(part) for part in command.split(' | ')]
    for i, cmd in enumerate(commands):
        if i==0:
            res = subprocess.Popen(cmd, stdout=subprocess.PIPE)
        else:
            res = subprocess.Popen(cmd, stdin=res.stdout, stdout=subprocess.PIPE)
    total, used = map(int, res.communicate()[0].decode('utf-8').strip().split('\n'))
    info = {'total_MiB':total, 'used_MiB':used}
    return info

# xtreme-vision

In [ ]:
# 1.51はバグる
!pip install xtreme-vision==1.4

     |████████████████████████████████| 256kB 12.4MB/s 


In [ ]:
%%time
# jpgの姿勢推定
from xtreme_vision.Estimation import Pose_Estimation
input_path = "/content/drive/MyDrive/00Colab Notebooks/07Datasets/video/hito.jpg"
output_path = "/content/hito.jpg".format("pose_")
model = Pose_Estimation()
model.Use_CenterNet()

--------------------
Loading the Model 
 Please Wait...
--------------------
781025280/781019000 [==============================] - 23s 0us/step
CPU times: user 13.2 s, sys: 3.09 s, total: 16.3 s
Wall time: 38.3 s


In [ ]:
%%time
model.Detect_From_Image(input_path=input_path, output_path=output_path)

CPU times: user 2.17 s, sys: 647 ms, total: 2.82 s
Wall time: 6.53 s


In [ ]:
%%time
# videoの姿勢推定
from xtreme_vision.Estimation import Pose_Estimation
input_path = "/content/drive/MyDrive/00Colab Notebooks/07Datasets/video/Man_Street.mp4"
output_path = "/content/{}video.mp4".format("pose_")
model = Pose_Estimation()
model.Use_CenterNet()

--------------------
Loading the Model 
 Please Wait...
--------------------
781025280/781019000 [==============================] - 22s 0us/step
CPU times: user 14.6 s, sys: 2.6 s, total: 17.2 s
Wall time: 35.2 s


In [ ]:
%%time
#fpsは画像からビデオを再構成するとき使用するfps＝小さいほど動画時間が伸びる（元ビデオから処理するフレーム数は固定）
model.Detect_From_Video(input_path=input_path, output_path=output_path,fps=25)


There are 216 Frames in this video
--------------------
Detecting Objects in the Video... Please Wait...
--------------------
Done. Processing has been Finished... Please Check Output Video.
CPU times: user 1min 6s, sys: 1.28 s, total: 1min 7s
Wall time: 1min 16s


In [ ]:
%%time
# jpgの物体検出
from xtreme_vision.Detection import Object_Detection
input_path = "/content/drive/MyDrive/00Colab Notebooks/07Datasets/video/hito.jpg"
output_path = "/content/{}hito.jpg".format("pose_")
model = Object_Detection()
model.Use_CenterNet()

Please Wait...
766869504/766861424 [==============================] - 22s 0us/step
CPU times: user 14.4 s, sys: 2.3 s, total: 16.7 s
Wall time: 35.5 s


In [ ]:
%%time
model.Detect_From_Image(input_path=input_path, output_path=output_path)

CPU times: user 423 ms, sys: 9.92 ms, total: 433 ms
Wall time: 452 ms


In [ ]:
%%time
# jpgの物体検出
from xtreme_vision.Detection import Object_Detection
input_path = "/content/drive/MyDrive/00Colab Notebooks/07Datasets/video/Man_Street.mp4"
output_path = "/content/{}video.mp4".format("pose_")
model = Object_Detection()
model.Use_CenterNet()

Please Wait...
CPU times: user 12.3 s, sys: 612 ms, total: 12.9 s
Wall time: 12.5 s


In [ ]:
%%time
#fpsは画像からビデオを再構成するとき使用するfps＝小さいほど動画時間が伸びる（元ビデオから処理するフレーム数は固定）
model.Detect_From_Video(input_path=input_path, output_path=output_path)


There are 216 Frames in this video
--------------------
Detecting Objects in the Video... Please Wait...
--------------------
Done. Processing has been Finished... Please Check Output Video.
CPU times: user 1min 2s, sys: 699 ms, total: 1min 3s
Wall time: 1min


# efficient-det

In [ ]:
%cd /content
!rm -rf /content/Yet-Another-EfficientDet-Pytorch
!git clone https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch.git
%cd /content/Yet-Another-EfficientDet-Pytorch

/content
Cloning into 'Yet-Another-EfficientDet-Pytorch'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 725 (delta 2), reused 7 (delta 2), pack-reused 717
Receiving objects: 100% (725/725), 8.60 MiB | 9.16 MiB/s, done.
Resolving deltas: 100% (417/417), done.
/content/Yet-Another-EfficientDet-Pytorch


In [ ]:
# install requirements
!pip install pycocotools numpy opencv-python tqdm tensorboard tensorboardX pyyaml webcolors
!pip install torch==1.4.0
!pip install torchvision==0.5.0

     |████████████████████████████████| 317kB 11.8MB/s 
     |████████████████████████████████| 753.4MB 21kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
     |████████████████████████████████| 4.0MB 12.9MB/s 
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [ ]:
!mkdir weights

In [ ]:
# run the simple inference script
!python efficientdet_test.py

running speed test...
test1: model inferring and postprocessing
inferring image for 10 times...
0.040840554237365725 seconds, 24.485465946127704 FPS, @batch_size 1


# yolov4

In [ ]:
%cd /content
!rm -rf /content/pytorch-YOLOv4
!git clone https://github.com/Tianxiaomo/pytorch-YOLOv4.git
%cd /content/pytorch-YOLOv4

Cloning into 'pytorch-YOLOv4'...
remote: Enumerating objects: 914, done.
remote: Total 914 (delta 0), reused 0 (delta 0), pack-reused 914
Receiving objects: 100% (914/914), 2.34 MiB | 4.40 MiB/s, done.
Resolving deltas: 100% (558/558), done.


In [ ]:
%cd /content
!wget https://drive.google.com/u/0/uc?export=download&confirm=pL8B&id=1fcbR0bWzYfIEdLJPzOsn4R5mlvR6IQyA


/content
--2020-12-08 02:00:45--  https://drive.google.com/u/0/uc?export=download
Resolving drive.google.com (drive.google.com)... 108.177.119.102, 108.177.119.101, 108.177.119.139, ...
Connecting to drive.google.com (drive.google.com)|108.177.119.102|:443... connected.
HTTP request sent, awaiting response... 400 Bad Request
2020-12-08 02:00:45 ERROR 400: Bad Request.



# yolov5
https://github.com/ultralytics/yolov5

In [1]:
!nvidia-smi

Fri Dec 25 02:55:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# %cd /content
# !rm -rf /content/yolov5
# !git clone https://github.com/ultralytics/yolov5
# %cd /content/yolov5

# %cd "/content/drive/MyDrive/00Colab Notebooks/07Datasets"
# !git clone https://github.com/ultralytics/yolov5
# %cd "/content/drive/MyDrive/00Colab Notebooks/07Datasets/yolov5"

In [7]:
%cd "/content/drive/MyDrive/00Colab Notebooks/07Datasets/yolov5"
!pip install -r re2.txt
# !pip install -r requirements.txt

/content/drive/MyDrive/00Colab Notebooks/07Datasets/yolov5
     |████████████████████████████████| 276kB 13.2MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=50ca4b64fdd65d67e0c5dd753bec34327dafd5d03721ff9bbd73a0543d509d9a
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [9]:
import os
# len(os.listdir("/content/drive/MyDrive/00Colab Notebooks/07Datasets/COCO/val2017"))
len(os.listdir("/content/drive/MyDrive/00Colab Notebooks/07Datasets/COCO/cocomax"))

512

In [10]:
import torch
torch.cuda.is_available()

True

In [11]:
!python test_kai.py --img-size 424 --weights yolov5s.pt --batch-size 32

乱数設定完了 False
2020-12-25 12:06:40,153 - INFO - logger set up
2020-12-25 12:06:40,154 - INFO - log:log_test
2020-12-25 12:06:40,175 - INFO - gpu_start{'total_MiB': 16130, 'used_MiB': 10}
data/coco128.yaml False
Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/coco128.yaml', device='', exist_ok=False, img_size=424, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['yolov5s.pt'])
2020-12-25 12:06:49,052 - INFO - arg:Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/coco128.yaml', device='', exist_ok=False, img_size=424, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['yolov5s.pt'])
2020-12-25 12:12:53,952 - INFO - log:512(500, 375, 3)
2020-12-25 12:12:53,964 - INFO - Using torch 1.7.0+cu101 CUDA:0 (Tesla V100-SX

In [ ]:
!python test_kai.py --img-size 848 --weights yolov5x.pt --batch-size 32

乱数設定完了 False
2020-12-15 13:24:26,765 - INFO - logger set up
2020-12-15 13:24:26,766 - INFO - log:log_test
2020-12-15 13:24:26,791 - INFO - gpu_start{'total_MiB': 16130, 'used_MiB': 10}
data/coco128.yaml False
Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/coco128.yaml', device='', exist_ok=False, img_size=848, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['yolov5x.pt'])
2020-12-15 13:24:27,370 - INFO - arg:Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/coco128.yaml', device='', exist_ok=False, img_size=848, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['yolov5x.pt'])
2020-12-15 13:24:42,874 - INFO - log:512(612, 612, 3)
2020-12-15 13:24:42,882 - INFO - Using torch 1.7.0+cu101 CUDA:0 (Tesla V100-SX

In [ ]:
!python test_kai.py --img-size 848 --weights yolov5x.pt --batch-size 32

乱数設定完了 False
2020-12-15 12:19:04,639 - INFO - logger set up
2020-12-15 12:19:04,640 - INFO - log:log_test
2020-12-15 12:19:04,662 - INFO - gpu_start{'total_MiB': 16130, 'used_MiB': 10}
data/coco128.yaml False
Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/coco128.yaml', device='', exist_ok=False, img_size=848, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['yolov5x.pt'])
2020-12-15 12:19:05,186 - INFO - arg:Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/coco128.yaml', device='', exist_ok=False, img_size=848, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['yolov5x.pt'])
2020-12-15 12:19:09,199 - INFO - log:500(612, 612, 3)
2020-12-15 12:19:09,207 - INFO - Using torch 1.7.0+cu101 CUDA:0 (Tesla V100-SX

In [ ]:
# imread
!python test_numpy.py --img-size 424 --weights yolov5x.pt --batch-size 32

乱数設定完了 False
2020-12-15 14:26:06,309 - INFO - logger set up
2020-12-15 14:26:06,310 - INFO - log:log_test
2020-12-15 14:26:06,335 - INFO - gpu_start{'total_MiB': 16130, 'used_MiB': 10}
data/coco128.yaml False
Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/coco128.yaml', device='', exist_ok=False, img_size=424, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['yolov5x.pt'])
2020-12-15 14:26:06,914 - INFO - arg:Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/coco128.yaml', device='', exist_ok=False, img_size=424, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['yolov5x.pt'])
2020-12-15 14:26:06,922 - INFO - Using torch 1.7.0+cu101 CUDA:0 (Tesla V100-SXM2-16GB, 16130MB)
2020-12-15 14:26:06,923 - INFO - 
Fu

In [ ]:
!python test_kai.py --img-size 848 --weights yolov5x.pt --batch-size 32
!python test_kai.py --img-size 848 --weights yolov5l.pt --batch-size 32
!python test_kai.py --img-size 848 --weights yolov5m.pt --batch-size 32
!python test_kai.py --img-size 848 --weights yolov5s.pt --batch-size 32
!python test_kai.py --img-size 424 --weights yolov5x.pt --batch-size 32
!python test_kai.py --img-size 424 --weights yolov5l.pt --batch-size 32
!python test_kai.py --img-size 424 --weights yolov5m.pt --batch-size 32
!python test_kai.py --img-size 424 --weights yolov5s.pt --batch-size 32

!python test_kai.py --img-size 848 --weights yolov5x.pt --batch-size 16
!python test_kai.py --img-size 848 --weights yolov5l.pt --batch-size 16
!python test_kai.py --img-size 848 --weights yolov5m.pt --batch-size 16
!python test_kai.py --img-size 848 --weights yolov5s.pt --batch-size 16
!python test_kai.py --img-size 424 --weights yolov5x.pt --batch-size 16
!python test_kai.py --img-size 424 --weights yolov5l.pt --batch-size 16
!python test_kai.py --img-size 424 --weights yolov5m.pt --batch-size 16
!python test_kai.py --img-size 424 --weights yolov5s.pt --batch-size 16

!python test_kai.py --img-size 848 --weights yolov5x.pt --batch-size 8
!python test_kai.py --img-size 848 --weights yolov5l.pt --batch-size 8
!python test_kai.py --img-size 848 --weights yolov5m.pt --batch-size 8
!python test_kai.py --img-size 848 --weights yolov5s.pt --batch-size 8
!python test_kai.py --img-size 424 --weights yolov5x.pt --batch-size 8
!python test_kai.py --img-size 424 --weights yolov5l.pt --batch-size 8
!python test_kai.py --img-size 424 --weights yolov5m.pt --batch-size 8
!python test_kai.py --img-size 424 --weights yolov5s.pt --batch-size 8

!python test_kai.py --img-size 848 --weights yolov5x.pt --batch-size 1
!python test_kai.py --img-size 848 --weights yolov5l.pt --batch-size 1
!python test_kai.py --img-size 848 --weights yolov5m.pt --batch-size 1
!python test_kai.py --img-size 848 --weights yolov5s.pt --batch-size 1
!python test_kai.py --img-size 424 --weights yolov5x.pt --batch-size 1
!python test_kai.py --img-size 424 --weights yolov5l.pt --batch-size 1
!python test_kai.py --img-size 424 --weights yolov5m.pt --batch-size 1
!python test_kai.py --img-size 424 --weights yolov5s.pt --batch-size 1


乱数設定完了 False
2020-12-15 13:26:26,039 - INFO - logger set up
2020-12-15 13:26:26,040 - INFO - log:log_test
2020-12-15 13:26:26,064 - INFO - gpu_start{'total_MiB': 16130, 'used_MiB': 10}
data/coco128.yaml False
Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/coco128.yaml', device='', exist_ok=False, img_size=848, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['yolov5x.pt'])
2020-12-15 13:26:26,626 - INFO - arg:Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/coco128.yaml', device='', exist_ok=False, img_size=848, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['yolov5x.pt'])
2020-12-15 13:26:30,986 - INFO - log:512(612, 612, 3)
2020-12-15 13:26:30,994 - INFO - Using torch 1.7.0+cu101 CUDA:0 (Tesla V100-SX

In [ ]:
%%time
#  !python detect.py --source "/content/drive/MyDrive/00Colab Notebooks/07Datasets/COCO/val2017" --weights yolov5s.pt

# !python detect_kai.py --source "/content/drive/MyDrive/00Colab Notebooks/07Datasets/COCO/cocomax" --img-size 848 --weights yolov5x.pt
# !python detect.py --source "/content/drive/MyDrive/00Colab Notebooks/07Datasets/COCO/cocomini" --img-size 424 --weights yolov5s.pt

#  !python detect.py --source "/content/drive/MyDrive/00Colab Notebooks/07Datasets/COCO/cocomini" --weights yolov5s.pt

乱数設定完了 False
2020-12-14 15:55:07,452 - INFO - logger set up
2020-12-14 15:55:07,453 - INFO - log:log_test
2020-12-14 15:55:07,479 - INFO - gpu_start{'total_MiB': 16130, 'used_MiB': 10}
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=848, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=False, source='/content/drive/MyDrive/00Colab Notebooks/07Datasets/COCO/cocomax', update=False, view_img=False, weights=['yolov5x.pt'])
2020-12-14 15:55:08,124 - INFO - Using torch 1.7.0+cu101 CUDA:0 (Tesla V100-SXM2-16GB, 16130MB)
2020-12-14 15:55:08,125 - INFO - 
Fusing layers... 
2020-12-14 15:55:14,365 - INFO - Model Summary: 484 layers, 88922205 parameters, 0 gradients, 221.5 GFLOPS
2020-12-14 15:55:14,739 - INFO - gpu_model{'total_MiB': 16130, 'used_MiB': 1953}
image 1/500 /content/drive/MyDrive/00Colab Notebooks/07Datasets/COCO/cocomax/000000000139.jpg: Traceback (most recent call last):
  File "detect_kai.